## We try to make a real script based on this electron gun spectrum 

This log is part of the data_analysis for 1st POLARIS XFEL beamtime at Desy, FLASH (Oct.2018)

(1) Now we are able to scan the energy to see the electron gun spectrum. The energy of the electron is set as, and the energy range for this scan is eV to eV. We try to use this HDF5 file to generate our first real script.

(2) There is still no Laser information in this HDF5 file.

----------- 
 Created by  Hsin-Yi Wang (wang.hy@fysik.su.se), Date: 01-Oct-2018
- Note: This script shuld be operated under Python 3 environment 

### Import modules and useful function

In [1]:
import numpy as np
import random
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
import h5py
import sys
from os import walk
import os.path
#from mpl_toolkits.axes_grid1 import make_axes_locatable

sys.path.insert(0, '../src/')
import fit_data_tools
from fit_data_tools import *

### Import parameters 

In [5]:
#######################
# Spectrum parameters #
#######################
Escan_Vi = 400 # Initial scanning energy
Escan_Vf = 402 # Final scanning energy
Ep = 100 # Passing energy
N_sweep = 10 # sweep number
files=[1,2] # File name, ex: files = [1] for XPS001. By now the number should be < 999 
Spectrum_name = ["C1s", "O1s"] # the name for this spectrum

######################
# Creating threshold #
######################
thr_camera = 3 # smallest blob number per pixel
thr_V = 10 # just use to filt out the data ponit where the voltage/scanning energy reading is zero  
#thr_Laser = 

### Start to do data analysis and polt out the filgure

In [9]:
####################################################
# Define the ratio between pixel and energy scale  #
####################################################
Pixels2V = 0.1654*Ep/1000 #  = how much V per pixel

######################
# Load the data file #
######################
i = 0
while i < len(files):
    data_path = '/Users/hsin-yiwang/Desktop/Insitucell_for_XPS/FlashBeamtime/DATA/'+'XPS%03d' %files[i]
    if not os.path.exists(data_path) :
        print 'XPS%03d does not exist! Check again if you put the right data path or not'%files[i]
    data_names = discover_datas(data_path)
    print (data_names)
    
    #############################################
    # Creating empty matirx for collecting data #
    #############################################

    Total_profile = []
    Extend_total_profile = []
    Total_V = []
    #Laser = []

    #Pump_filter = []
    #Nonpump_filter = []

    ##Pump_Total_profile = []
    #Pump_Total_V = []
    #Pump_Extend_total_profile = []

    #Nonpump_Total_profile = []
    #Nonpump_Total_V = []
    #Nonpump_Extend_total_profile = []

    ################
    # Data process #
    ################
    for i in range(len(data_names)):
        h5file = h5py.File(data_path+data_names[i], 'r')
        Image = h5file['/FL1/Experiment/Camera/PG/Basler FW 1/image/'].value
        #Ind_profile = h5file['/FL1/Experiment/Camera/PG/Basler FW 1/ROI.Y/'].value #The summed individule image along y 
        Voltage_num = h5file['/FL1/Experiment/PG/SIS8300 100MHz ADC/CH6/TD/'].value
        #Laser_reading = h5file['/FL1/Experiment/PG/_______________________'].value
    
        Image[Image<thr_camera] = 0
        Ind_profile = np.sum(Image[i,:,:],axis = 0)
        Total_profile.append(Ind_profile)
    
        Ind_voltage_num = np.average(Voltage_num,axis = 1)
        Ind_V = Ind_Voltage_num*650/2**16
        Total_V.append(Ind_V)
        
        """
        ## need to check how to access the Laser reading and how it looks like in the HDF5 file
        Laser.append(Laser_reading)  
        """

    Total_profile = np.array(Total_profile)
    Total_V = np.array(Voltage_V)
    #Laser_reading = np.array(Laer_reading)

    Mask_Voltage = Total_V<thr_V
    Total_profile = Total_profile[Mask_Voltage,:]
    Total_V = Total_V[Mask_Voltage]

    """
    Pump_filter.append(Laser>thr_Laser)
    Pump_filter.append(Total_V<thr_V)
    Pump_mask = np.all(Pump_filter, axis=0)
    Pump_Total_profile = Total_profile[Pump_mask,:]
    Pump_Total_V = Total_V[Pump_mask]

    Nonpump_filter.append(Laser<thr_Laser)
    Nonpump_filter.append(Total_V<thr_V)
    Nonpump_mask = np.all(Nonpump_filter, axis=0)
    Nonpump_Total_profile = Total_profile[Nonump_mask,:]
    Nonpump_Total_V = Total_V[Nonump_mask]
    """

    ############################################
    # Convert all energy scale to pixels scale #
    ############################################

    AssFull_pixels_range = np.ceil((Escan_Vf-Escan_Vi)*1.05/float(Pixels2V))
    AssReal_Escan_Vi = Ecan_Vi-(Escan_Vf-Escan_Vi)*0.025
    AssReal_Escan_Vf = Ecan_Vf+(Escan_Vf-Escan_Vi)*0.025
    Check_AssReal_Escan_Vf = AssReal_Ecan_Vi+(AssFull_pixels_range)*float(Pixels2V)

    print ('Check the pixels2V range by two differnt methods for XPS%03d' %files[i])
    print ('AssReal_Escan_Vf = %0.3f'%AssReal_Escan_Vf)
    print ('Check_AssReal_Escan_Vf = %0.3f'%Check_AssReal_Escan_Vf)

    for j in np.arange(Total_profile.shape[0]):   
        Profile_center = np.ceil((Total_V[j]-AssReal_Escan_Vi)/float(Pixels2V))
        Extend_profile = np.concatenate([np.zeros(Profile_center-240), Total_profile[j,:], np.zeros(AssFull_pixels_range-Profile_center-240)])
        Extend_total_profile.append(Extend_profile)

    Extend_total_profile = np.array(Extend_total_profile)

    #################################################
    # Plot out the data in the desired energy range #
    #################################################
    Spectrum = np.sum(Extend_total_profile,axis = 0)/N_sweep
    E_scale = np.arange(AssFull_pixels_range)*float(Pixels2V)+AssReal_Escan_Vi
    SizeSpectrum = Spectrum.shape[0]
    SizeE_scale = E_scale.shape[0]
    if SizeSpectrum != SizeE_scale:
        print("The Spectrum size is not comparable with the Energy_scale size !!")
    
    Frame = np.all(E_scale >= Escan_Vi, E_scale <= Escan_Vf, axis=0)
    Spectrum = Sepctrum[Frame]
    E_scale = E_scale[Frame]
                                         
    plt.figure(figsize=[10,8])
    plt.plot(E_scale,Spectrom,ls = '-',linewidth ='2', c='red', label = 'Spectrum = %s' %Spectrum_name[i])
    plt.xlabel('Kinetic energy (eV)',size=12)
    plt.ylabel('Normalized counts (a. u.)',size=12)
    plt.title('Electron gun spectrum',size=12)
    plt.grid()
    plt.tight_layout()
    plt.legend(frameon=False,loc='upper right', prop={'size': 12})
    plt.show()

    ###########################################
    # Save the data.txt in the same file path #
    ###########################################
    TXT_save_path = '/Users/hsin-yiwang/Desktop/Insitucell_for_XPS/FlashBeamtime/DATA/'+'XPS%03d' %files[i]
    completeName = os.path.join(TXT_save_path, "Sepctrum_"+"XPS%03d"+".txt" %(files[i]))         
    with open (completeName, "w") as out_file:
        for k in range(len(E_scale)):
            out_string = ""
            out_string += str(E_scale[k])
            out_string += ","+ str(Spectrum[k])
            out_string += "\n"
            out_file.write(out_string)
    i = i+1

The full range pixel numbers = 121
/Users/hsin-yiwang/Desktop/Insitucell_for_XPS/FlashBeamtime/DATA/XPS001
